# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import ttest_1samp
from scipy.stats import sem
from statsmodels.stats.proportion import proportions_ztest

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [2]:
# Run this code:

salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
# Your code here:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


In [4]:
salaries.shape

(33183, 8)

We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [5]:
# Your code here:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [6]:
salaried_employees = salaries['Full or Part-Time'].count()
salaried_employees

33183

In [7]:
hourly_employees = salaries['Hourly Rate'].count()
hourly_employees

8022

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [8]:
salaries.Department.value_counts()

Department
POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [9]:
hourly = salaries[salaries['Hourly Rate'].notna()]
hourly_col = hourly['Hourly Rate']

# Perform one-sample t-test
t_statistic, p_value = stats.ttest_1samp(hourly_col, 
                                         30)
alpha = 0.05  # significance level

# Compare the p-value with alpha for a two-tailed test
if p_value < alpha:
    print(f"Reject the null hypothesis. p-value:{p_value} The hourly wage of all hourly workers is significantly different from $30/hr.")
else:
    print("Fail to reject the null hypothesis. There is not enough evidence to conclude that the hourly wage is significantly different from $30/hr.")

Reject the null hypothesis. p-value:4.3230240486229894e-92 The hourly wage of all hourly workers is significantly different from $30/hr.


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [10]:
annual_salaries = salaries[(salaries['Annual Salary'].notna()) & (salaries['Department'] == 'POLICE')]
police_salaries = annual_salaries['Annual Salary']

last_year_mean = 86000  # mean salary last year

t_statistic, p_value = stats.ttest_1samp(police_salaries,
                                         last_year_mean,
                                         alternative='less')

alpha = 0.05  # significance level

# Check if the p-value is less than alpha
if p_value < alpha:
    print("Reject the null hypothesis. There is evidence that police salaries this year are less than last year's mean.")
else:
    print("Fail to reject the null hypothesis. There is not enough evidence to conclude that police salaries this year are less than last year's mean.")


Fail to reject the null hypothesis. There is not enough evidence to conclude that police salaries this year are less than last year's mean.


Using the `crosstab` function, find the department that has the most hourly workers. 

In [11]:
contingency_table = pd.crosstab(salaries['Department'], salaries['Salary or Hourly'])
hourly_counts = contingency_table['Hourly']
max_hourly_department = hourly_counts.idxmax()

print(f"\nThe department with the most hourly workers is: {max_hourly_department}")



The department with the most hourly workers is: STREETS & SAN


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [12]:
most_hourly_department = hourly_counts.idxmax()

# Filter data for the identified department and non-null hourly wages
hourly_wages_department = salaries[(salaries['Department'] == most_hourly_department) & (salaries['Hourly Rate'].notna())]['Hourly Rate']

# Given data
expected_hourly_rate = 35  # Expected hourly rate for comparison

# Perform one-sample t-test
t_statistic, p_value = stats.ttest_1samp(hourly_wages_department,
                                          expected_hourly_rate,
                                          alternative='less')

# Hypothesis test
alpha = 0.05  # significance level

if p_value < alpha:
    print(f"Reject the null hypothesis. There is evidence that the hourly wage in the '{most_hourly_department}' department is less than $35/hour.")
else:
    print(f"Fail to reject the null hypothesis. There is not enough evidence to conclude that the hourly wage in the '{most_hourly_department}' department is less than $35/hour.")

Reject the null hypothesis. There is evidence that the hourly wage in the 'STREETS & SAN' department is less than $35/hour.


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [13]:
df = salaries[salaries['Salary or Hourly']== 'Hourly']

# Your code here:
stats.t.interval(0.95,                  # alfa = 1 - pvalor, confianza
           len(df)-1,      # N-1, dof
           loc=df['Hourly Rate'].mean(),
           scale=sem(df['Hourly Rate'])
          )

(32.52345834488425, 33.05365708767623)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [14]:
df2 = salaries[(salaries['Salary or Hourly']== 'Salary') & (salaries['Department']== 'POLICE') | (salaries['Department']== 'POLICE BOARD')]

stats.t.interval(0.95,                  
           len(df2)-1,      
           loc=df2['Annual Salary'].mean(),
           scale=sem(df2['Annual Salary'])
          )

(86177.02348322631, 86795.70096851172)

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [15]:
from statsmodels.stats.proportion import proportions_ztest

z_score, p_value= proportions_ztest(salaries['Salary or Hourly'].value_counts()[1], 
                                             len(salaries),                     
                                               0.25                              
                                                   )
z_score, p_value

/var/folders/f2/ty6sgcw911g2_3vqhhq561vm0000gn/T/ipykernel_40667/427948105.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  z_score, p_value= proportions_ztest(salaries['Salary or Hourly'].value_counts()[1],


(-3.5099964213703005, 0.0004481127249057967)

- Z-Score: The Z-score is approximately `-3.51`. This value indicates how many standard deviations the observed proportion is from the expected value under the null hypothesis. A negative Z-score suggests that the observed proportion is lower than expected under the null hypothesis.

- P-Value: The p-value is very small, `0.000448`. This value indicates the probability of observing the observed proportion (or a more extreme proportion) if the true proportion is equal to 0.25. A small p-value suggests that you have significant evidence to reject the null hypothesis.